# OTIMIZAÇÃO DAS DESPESAS COM MARKETING

O seguinte trabalho tem como objetivo dar soluções de como podemos traçar melhores estratégias com o nosso marketing dado o nosso estado atual.
Portanto primeiro precisamos compreender qual o nosso estado atual, e depois estabelecermos as ações 

In [3]:
#importando bibliotecas

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [9]:
#Aqui estarão escritas todas as funções que serão utilizadas no código principal

def snake_case(df):
    """
    Função que transforma o nome das colunas de um DataFrame em snake_case
    """
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    return df

In [5]:
#primeira vista do primeiro df

df = pd.read_csv('datasets/visits_log_us.csv', nrows=100)
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Device     100 non-null    object
 1   End Ts     100 non-null    object
 2   Source Id  100 non-null    int64 
 3   Start Ts   100 non-null    object
 4   Uid        100 non-null    uint64
dtypes: int64(1), object(3), uint64(1)
memory usage: 4.0+ KB
None


,Device,End Ts,Source Id,Start Ts,Uid
0,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00,16879256277535980062
1,desktop,2018-02-19 17:21:00,2,2018-02-19 16:53:00,104060357244891740
2,touch,2017-07-01 01:54:00,5,2017-07-01 01:54:00,7459035603376831527
3,desktop,2018-05-20 11:23:00,9,2018-05-20 10:59:00,16174680259334210214
4,desktop,2017-12-27 14:06:00,3,2017-12-27 14:06:00,9969694820036681168


### visits

O primeiro df vai nos mostrar os acessos ao site. Ele nos responde quem é o usuário, como que entrou, quando e a origem do anúncio que o trouxe.
Devemos:

    1- Converter as colunas de data para datetime, que será fundamental para analisarmos esses dados
    2 - Converter para category os dados em "device".
    3 - Seguir as normas que exigem o snake_case. No caso todas as colunas renomeadas em minúsculas e com underline.

O nome do df será visits

In [7]:
print(df['Device'].value_counts())

Device
desktop    73
touch      27
Name: count, dtype: int64


In [10]:
visits = pd.read_csv('datasets/visits_log_us.csv', dtype={'Device':'category'}, parse_dates=['End Ts','Start Ts'])
visits = snake_case(visits)
print(visits.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359400 entries, 0 to 359399
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   device     359400 non-null  category      
 1   end_ts     359400 non-null  datetime64[ns]
 2   source_id  359400 non-null  int64         
 3   start_ts   359400 non-null  datetime64[ns]
 4   uid        359400 non-null  uint64        
dtypes: category(1), datetime64[ns](2), int64(1), uint64(1)
memory usage: 11.3 MB
None
